# Testing baseline and finetuned model v1 and v2
use batch API to test the performance of baseline and finetuned models with the testing data
this procedure is faster than getting the performance synchronously

In [ ]:
from openai import OpenAI
import os
import json
import random
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#client = OpenAI()

# Testing the baseline model
1. create the jsonl file for submission
2. submit to batch API
3. extract prediction from returned result
4. extract target from validation.jsonl
5. get the report

In [23]:
#batch file for baseline model
# Input file with Greek words (one word per line)
input_file = 'validation.jsonl'  
output_file = 'base_requests.jsonl' 
greek_words= []

# Load Greek words from input file
with open(input_file, 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        messages = data.get("messages", [])
        words = [msg["content"] for msg in messages if msg["role"] == "user"]
        greek_words.extend(words)

# Generate JSONL data
with open(output_file, 'w', encoding='utf-8') as outfile:
    for i, word in enumerate(greek_words, start=1):  # Start custom_id from 1
        request = {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {
                        "role": "system",
                        "content": "Classify using one label: O, B-PERSON, B-NORP, B-LOC, B-GOD, B-GPE, I-LOC, I-PERSON, I-GOD, B-LANGUAGE, B-ORG, I-LANGUAGE, I-NORP, B-WORK, B-EVENT."
                    },
                    {
                        "role": "user",
                        "content": word
                    }
                ],
                "max_tokens": 5,
                "temperature": 0
            }
        }
        # Write JSON object as a line in JSONL file
        outfile.write(json.dumps(request) + '\n')

print(f"JSONL file '{output_file}' created successfully!")


JSONL file 'base_requests.jsonl' created successfully!


In [25]:
# client.files.create(
#   file=open("base_requests.jsonl", "rb"),
#   purpose="batch"
# )

FileObject(id='file-TNTRFdv8OW26wWyQxniw22mo', bytes=5428664, created_at=1731962606, filename='base_requests.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [29]:
# client.batches.create(
#   input_file_id="file-TNTRFdv8OW26wWyQxniw22mo",
#   endpoint="/v1/chat/completions",
#   completion_window="24h"
# )

Batch(id='batch_673ba70207388190b34a4d070f606e93', completion_window='24h', created_at=1731962626, endpoint='/v1/chat/completions', input_file_id='file-TNTRFdv8OW26wWyQxniw22mo', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1732049026, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [15]:
client.batches.retrieve("batch_673ba70207388190b34a4d070f606e93")

Batch(id='batch_673ba70207388190b34a4d070f606e93', completion_window='24h', created_at=1731962626, endpoint='/v1/chat/completions', input_file_id='file-TNTRFdv8OW26wWyQxniw22mo', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1731981202, error_file_id=None, errors=None, expired_at=None, expires_at=1732049026, failed_at=None, finalizing_at=1731979689, in_progress_at=1731962689, metadata=None, output_file_id='file-XyJ5rwvcqwWFtxrSWFnRZyy7', request_counts=BatchRequestCounts(completed=13414, failed=0, total=13414))

In [33]:
y_true = []  # List to store the actual labels
# Extract and store the target value
with open("validation.jsonl", "r", encoding="utf-8") as file:
    for line in file:
        data = json.loads(line)
    
        for message in data.get("messages", []):
            if message["role"] == "assistant":
                target_label = message["content"]
                y_true.append(target_label)

In [27]:
file_response = client.files.content("file-XyJ5rwvcqwWFtxrSWFnRZyy7")
file_content = file_response.text  # Get the text content from the response

# Save the content to a file
with open("base_results.txt", "w", encoding="utf-8") as file:
    file.write(file_content)

print("File saved successfully as 'base_results.txt'")
file_path = "base_results.txt"  # Replace with your file's path
result = verify_sequential_custom_id(file_path)
print(result)

File saved successfully as 'base_results.txt'
All custom_id values are sequential.


In [29]:
y_pred = []  # To store the extracted content

with open("base_results.txt", 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        try:
            # Parse the JSON object from the line
            data = json.loads(line.strip())

            # Navigate to the desired field
            content = data["response"]["body"]["choices"][0]["message"]["content"]
            y_pred.append(content)

        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error parsing line {line_number}: {e}")

In [35]:
#Then we calculate precision, recall, F1-score, and support
base_report = classification_report(y_true, y_pred, labels=[
    'O', 'B-PERSON', 'B-NORP', 'B-LOC', 'B-GOD', 'B-GPE', 'I-LOC', 'I-PERSON', 'I-GOD',
    'B-LANGUAGE', 'B-ORG', 'I-LANGUAGE', 'I-NORP', 'B-WORK', 'B-EVENT'])
print("Classification Report of the baseline model:")
print(base_report)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sk

Classification Report of the baseline model:
              precision    recall  f1-score   support

           O       1.00      0.92      0.96     12437
    B-PERSON       0.63      0.75      0.69       454
      B-NORP       0.66      0.52      0.58       300
       B-LOC       0.35      0.69      0.47       152
       B-GOD       0.05      0.83      0.09        48
       B-GPE       0.00      0.00      0.00         7
       I-LOC       0.00      0.00      0.00         6
    I-PERSON       0.00      0.00      0.00         4
       I-GOD       0.00      0.00      0.00         2
  B-LANGUAGE       0.00      0.00      0.00         2
       B-ORG       0.00      0.00      0.00         1
  I-LANGUAGE       0.00      0.00      0.00         1
      I-NORP       0.00      0.00      0.00         0
      B-WORK       0.00      0.00      0.00         0
     B-EVENT       0.00      0.00      0.00         0

   micro avg       0.90      0.90      0.90     13414
   macro avg       0.18      0.25  

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Testing the finetuned model
1. create the jsonl file for submission
2. submit to batch API
3. extract prediction from returned result
4. extract target from validation.jsonl
5. get the report

In [3]:
# batch file for finetuned model
# Input file with Greek words (one word per line)
input_file = 'validation.jsonl'  
output_file = 'ft_v2_requests.jsonl' 
greek_words= []

# Load Greek words from input file
with open(input_file, 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        messages = data.get("messages", [])
        words = [msg["content"] for msg in messages if msg["role"] == "user"]
        greek_words.extend(words)

# Generate JSONL data
with open(output_file, 'w', encoding='utf-8') as outfile:
    for i, word in enumerate(greek_words, start=1):  # Start custom_id from 1
        request = {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "ft:gpt-4o-mini-2024-07-18:personal::AV9yQ9BO:ckpt-step-1016",
                "messages": [
                    {
                        "role": "system",
                        "content": "Classify using one label: O, B-PERSON, B-NORP, B-LOC, B-GOD, B-GPE, I-LOC, I-PERSON, I-GOD, B-LANGUAGE, B-ORG, I-LANGUAGE, I-NORP, B-WORK, B-EVENT."
                    },
                    {
                        "role": "user",
                        "content": word
                    }
                ],
                "max_tokens": 5,
                "temperature": 0
            }
        }
        # Write JSON object as a line in JSONL file
        outfile.write(json.dumps(request) + '\n')

print(f"JSONL file '{output_file}' created successfully!")


JSONL file 'ft_v2_requests.jsonl' created successfully!


In [7]:
# client.files.create(
#   file=open("ft_v2_requests.jsonl", "rb"),
#   purpose="batch"
# )

FileObject(id='file-f57phfTHNRcsGSELgefaHlTw', bytes=6072536, created_at=1732025024, filename='ft_v2_requests.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [11]:
# client.batches.create(
#   input_file_id="file-f57phfTHNRcsGSELgefaHlTw",
#   endpoint="/v1/chat/completions",
#   completion_window="24h"
# )

Batch(id='batch_673c9ae1777c8190b8ea734f694198d1', completion_window='24h', created_at=1732025057, endpoint='/v1/chat/completions', input_file_id='file-f57phfTHNRcsGSELgefaHlTw', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1732111457, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [39]:
client.batches.retrieve("batch_673c9ae1777c8190b8ea734f694198d1")

Batch(id='batch_673c9ae1777c8190b8ea734f694198d1', completion_window='24h', created_at=1732025057, endpoint='/v1/chat/completions', input_file_id='file-f57phfTHNRcsGSELgefaHlTw', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1732028871, error_file_id=None, errors=None, expired_at=None, expires_at=1732111457, failed_at=None, finalizing_at=1732027399, in_progress_at=1732025060, metadata=None, output_file_id='file-d70IHJ0lWbcIzAP33zkViIq4', request_counts=BatchRequestCounts(completed=13414, failed=0, total=13414))

In [41]:
file_response = client.files.content("file-d70IHJ0lWbcIzAP33zkViIq4")
file_content = file_response.text  # Get the text content from the response

# Save the content to a file
with open("ft_v2_results.txt", "w", encoding="utf-8") as file:
    file.write(file_content)

print("File saved successfully as 'ft_v2_results.txt'")

File saved successfully as 'ft_v2_results.txt'


In [25]:
#check the result follow the sequential order (that the previous order is not messed up)
def verify_sequential_custom_id(file_path):
    previous_request_number = None  # To keep track of the last request number

    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                # Parse each line as JSON
                data = json.loads(line.strip())

                # Extract the `custom_id` value and its numeric part
                custom_id = data.get("custom_id", "")
                if not custom_id.startswith("request-"):
                    raise ValueError(f"Invalid custom_id format at line {line_number}: {custom_id}")
                
                current_request_number = int(custom_id.split("-")[1])

                # Check if the current request number follows sequential order
                if previous_request_number is not None and current_request_number != previous_request_number + 1:
                    return f"Non-sequential custom_id at line {line_number}: {custom_id}"

                # Update the previous request number
                previous_request_number = current_request_number

            except (json.JSONDecodeError, ValueError) as e:
                return f"Error parsing line {line_number}: {str(e)}"

    return "All custom_id values are sequential."

In [43]:
# Example usage
file_path = "ft_v2_results.txt"  # Replace with your file's path
result = verify_sequential_custom_id(file_path)
print(result)

All custom_id values are sequential.


In [45]:
y_pred = []  # To store the extracted content

with open("ft_v2_results.txt", 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        try:
            # Parse the JSON object from the line
            data = json.loads(line.strip())

            # Navigate to the desired field
            content = data["response"]["body"]["choices"][0]["message"]["content"]
            y_pred.append(content)

        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error parsing line {line_number}: {e}")

In [47]:
#Then we calculate precision, recall, F1-score, and support
ft_report = classification_report(y_true, y_pred, labels=[
    'O', 'B-PERSON', 'B-NORP', 'B-LOC', 'B-GOD', 'B-GPE', 'I-LOC', 'I-PERSON', 'I-GOD',
    'B-LANGUAGE', 'B-ORG', 'I-LANGUAGE', 'I-NORP', 'B-WORK', 'B-EVENT'])
print("Classification Report of the ft model:")
print(ft_report)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report of the ft model:
              precision    recall  f1-score   support

           O       1.00      0.99      0.99     12437
    B-PERSON       0.90      0.96      0.93       454
      B-NORP       0.85      0.89      0.87       300
       B-LOC       0.77      0.86      0.81       152
       B-GOD       0.86      0.77      0.81        48
       B-GPE       1.00      0.43      0.60         7
       I-LOC       0.01      0.17      0.02         6
    I-PERSON       0.08      0.25      0.12         4
       I-GOD       0.29      1.00      0.44         2
  B-LANGUAGE       0.40      1.00      0.57         2
       B-ORG       1.00      1.00      1.00         1
  I-LANGUAGE       0.00      0.00      0.00         1
      I-NORP       0.00      0.00      0.00         0
      B-WORK       0.00      0.00      0.00         0
     B-EVENT       0.00      0.00      0.00         0

    accuracy                           0.98     13414
   macro avg       0.48      0.55      0.

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
